In [2]:
from pycaret.regression import *

In [52]:
import pandas as pd  
import numpy as np 
import itertools
import copy 
import matplotlib.pyplot as plt
import matplotlib as mpl

# Library Settings
mpl.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우에서 사용되는 한글 폰트
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스 기호가 깨지지 않도록 설정

TARGET_COL = 'next_1_price'
CROP = 'kimchi'
MODEL_NUM = 1
LIVE_YEAR = 2023
TEST_YEARS = [2022, 2021]

In [53]:
from sklearn.linear_model import LassoCV, RidgeCV, LarsCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from collections import Counter
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import mutual_info_regression

def split_data(my_data):
    """데이터를 학습 및 검증 세트로 나누는 함수"""
    for df_index in list(itertools.combinations(my_data['Year'].unique(), 2)):
        val_df_index = list(df_index)
        train_df_index = list(set(my_data['Year'].values) - set(val_df_index))
        val = my_data[my_data['Year'].isin(val_df_index)]
        train = my_data[my_data['Year'].isin(train_df_index)]
        yield val, train
        
def get_feature_importance(x, y, random_state=42,top_k=10):
    """
    다양한 모델(Lasso, Ridge, LARS, RandomForest)을 사용해 Feature Importance를 계산하고
    공통적으로 중요한 피처를 반환하는 함수입니다.
    
    Parameters:
        X (pd.DataFrame): 독립 변수 데이터프레임
        y (pd.Series): 종속 변수 (타겟)
        random_state (int): 난수 시드
    
    Returns:
        selected_features (list): 공통적으로 중요한 피처 목록
    """
    # 피처 목록
    feature_lists = []

    # 1. Lasso 모델
    lasso = LassoCV(cv=5, random_state=random_state).fit(x, y)
    lasso_importance = np.abs(lasso.coef_)
    lasso_features = x.columns[lasso_importance > 0].tolist()
    feature_lists.extend(lasso_features)
    print("Lasso Selected Features:", lasso_features)

    # 2. Ridge 모델
    ridge = RidgeCV(cv=5).fit(x, y)
    ridge_importance = np.abs(ridge.coef_)
    ridge_features = x.columns[ridge_importance > np.mean(ridge_importance)].tolist()
    feature_lists.extend(ridge_features)
    print("Ridge Selected Features:", ridge_features)

    # 3. LARS 모델
    lars = LarsCV(cv=5).fit(x, y)
    lars_importance = np.abs(lars.coef_)
    lars_features = x.columns[lars_importance > 0].tolist()
    feature_lists.extend(lars_features)
    print("LARS Selected Features:", lars_features)

    # 피처 등장 횟수 계산 
    feature_counts = Counter(feature_lists)
    
    # 세 가지 이상 모델에서 등장한 피처만 선택
    common_features = [feature for feature, count in feature_counts.items() if count >= 2][:top_k]

    return common_features 

def scale_data(data, type = "ss", scaler=None, is_fit=True): 
    if is_fit:
        """스케일링 함수 선정"""
        if type == "ss":
            scaler = StandardScaler()
        elif type == "ms":
            scaler = MinMaxScaler()
        elif type == "rs":
            scaler = RobustScaler()
        else: 
            print("Wrong Name of Scaler")
        X_scaled = scaler.fit_transform(data)
    else:
        scaler = scaler
        X_scaled = scaler.transform(data)

    # 스케일링된 데이터를 DataFrame으로 변환
    X_scaled_df = pd.DataFrame(X_scaled, columns=data.columns)
    
    return X_scaled_df, scaler

def prepare_data_for_prediction(data_seen, data_unseen):
    """
    주어진 데이터를 전처리하여 NaN을 제거하고, 예측에 사용할 데이터 준비.
    """
    test_data_1 = pd.DataFrame(data_unseen.loc[TEST_YEARS[0]]).T
    test_data_2 = pd.DataFrame(data_unseen.loc[TEST_YEARS[1]]).T
    #test_data_3 = pd.DataFrame(data_unseen.loc[TEST_YEARS[2]]).T
    data_2023 = pd.DataFrame(data_unseen.loc[2023]).T

    # NaN 제거
    test_data_1.drop(data_seen.columns[data_seen.isna().any()], axis=1, inplace=True)
    test_data_2.drop(data_seen.columns[data_seen.isna().any()], axis=1, inplace=True)
    #test_data_3.drop(data_seen.columns[data_seen.isna().any()], axis=1, inplace=True)
    data_2023.drop(data_seen.columns[data_seen.isna().any()], axis=1, inplace=True)

    data_unseen = data_unseen.drop(data_seen.columns[data_seen.isna().any()], axis=1)
    data_seen = data_seen.drop(data_seen.columns[data_seen.isna().any()], axis=1)
    
    data_seen.to_csv(f"data_seen_{CROP}{MODEL_NUM}.csv",encoding='utf-8-sig',index=None)
    data_unseen.to_csv(f"data_unseen_{CROP}{MODEL_NUM}.csv",encoding='utf-8-sig',index=None)
    data_2023.to_csv(f"data_2023_{CROP}{MODEL_NUM}.csv",encoding='utf-8-sig',index=None)

    return data_seen, data_unseen, test_data_1, test_data_2, data_2023 # test_data_3

def calculate_error_rate(actual, predicted):
    """평균 절대 오차율 계산"""
    return 100 * mean_absolute_error(actual, predicted) / np.mean(actual)


def create_voting_regressor(base_models_params, random_seed=None):
    estimators = []
    for name, model in base_models_params:
        if hasattr(model, 'random_state') and model.random_state is None:
            model.random_state = random_seed  # 랜덤 시드 설정
        estimators.append((name, model))  # VotingRegressor에 필요한 형식
    return VotingRegressor(estimators=estimators)

def train_and_predict(model, train_data, val_data, test_data, target_col):
    """모델 학습 및 예측"""
    # 모델의 `estimators` 속성을 확인
    if hasattr(model, 'estimators') and (not model.estimators or not isinstance(model.estimators, list)):
        raise ValueError("The 'estimators' attribute of the model is invalid. It should be a non-empty list of (string, estimator) tuples.")
    
    model.fit(train_data.drop(columns=[target_col, 'Year']), train_data[target_col])
    predictions_train = model.predict(train_data.drop(columns=[target_col, 'Year']))
    predictions_val = model.predict(val_data.drop(columns=[target_col, 'Year']))
    predictions_test = model.predict(test_data.drop(columns=[target_col, 'Year']))
    return predictions_train, predictions_val, predictions_test

def analyze_results(result_df, test_years, top_k=3):
    result_df['val_years_str'] = result_df['val_years'].astype(str)
    
    """결과 데이터프레임을 분석하여 최종 평균값 계산"""
    grouped_results = (
        result_df.groupby("val_years_str", group_keys=False)
        .apply(lambda x: x.nsmallest(top_k, "val"))
    )
    return {
        "train": grouped_results["train"].mean(),
        "val": grouped_results["val"].mean(),
        f"final_preds_{test_years[0]}": grouped_results[f"test_preds_{test_years[0]}"].mean(),
        f"final_preds_{test_years[1]}": grouped_results[f"test_preds_{test_years[1]}"].mean(),
        f"final_preds_{LIVE_YEAR}": grouped_results[f"live_preds_{LIVE_YEAR}"].mean(),
        "live_err": grouped_results["live_err"].mean(),
    }
    
def train_and_optimize_models(data_seen, model_num = 20, top_k = None, my_model_name = "my_model"):
    """
    주어진 학습 데이터로 모델을 설정하고 최적화하여 가장 좋은 모델을 반환.
    """
    data_seen = copy.deepcopy(data_seen)
    data_seen = data_seen.reset_index(drop=True)
    model_performance = []
    model_list = []

    for id in range(11, 11+model_num):  # 앙상블 모델 20개
        #id = random.randint(1,100)
        reg_test = setup(data=data_seen,
                         target=TARGET_COL,
                         use_gpu=True,
                         fold=3,
                         train_size=0.80,
                         session_id=id)

        # 모델 비교 및 최적화
        best_models = compare_models(
            sort='rmse',
            n_select=1,
        )

        model_name = best_models.__class__.__name__
        print(model_name)
        
        # 모델의 RMSE 추출
        model_rmse = pull()["RMSE"].iloc[0]
        model_performance.append((best_models, model_rmse))
        model_list.append(best_models)

    # RMSE 기준으로 정렬
    sorted_model_performance = sorted(model_performance, key=lambda x: x[1])
    
    # 정렬된 모델 리스트 (성능이 좋은 순서대로)
    model_list_sorted = [model[0] for model in sorted_model_performance]

    if top_k is None: # 별도로 갯수 옵션을 안줄시, 모델리스트의 모든 모델을 앙상블에 활용
        top_k = len(model_list) 
        
    # 모델 블렌딩
    result = blend_models(model_list_sorted[:top_k])  #  전체사용 
    result = finalize_model(result)  # 최종 모델로 고정    
    
    if my_model_name:
        save_model(model=result,
            model_name=my_model_name,
            verbose=False)
        
    return result 

def predict_with_models(model, data_seen, data_unseen):
    """
    주어진 모델들을 사용하여 예측하고, 예측 결과를 반환.
    """
    data_seen = copy.deepcopy(data_seen).reset_index(drop=True)
    data_unseen = copy.deepcopy(data_unseen).reset_index(drop=True)
    
    predictions_seen = predict_model(model, data=data_seen)
    predictions_seen["error_rate"] = 100 * (predictions_seen["prediction_label"] - predictions_seen[TARGET_COL]) / predictions_seen[TARGET_COL]
    seen_avg_error_rate = np.mean(np.abs(np.array(predictions_seen["error_rate"].iloc[:])))

    predictions_unseen = predict_model(model, data=data_unseen)
    predictions_unseen["error_rate"] = 100 * (predictions_unseen["prediction_label"] - predictions_unseen[TARGET_COL]) / predictions_unseen[TARGET_COL]
    unseen_avg_error_rate = np.mean(np.abs(np.array(predictions_unseen["error_rate"].iloc[:])))

    return predictions_seen, predictions_unseen, seen_avg_error_rate, unseen_avg_error_rate

def split_x_y(data, test_years):
    X = data.drop(columns=data.columns[data.columns.str.contains('next')],axis=1)  # 독립 변수만 모음
    X.dropna(axis=1, inplace=True) # 모든년도에서 쓸 수 있는 피처만 남기기
    y = data[TARGET_COL]  # 종속 변수
    
    X_real = X[X['year']==LIVE_YEAR] # 라이브 데이터는 따로 빼둔다
    y_real = y.loc[(X[X['year']==LIVE_YEAR]).index]
    
    X = X[X['year']!=LIVE_YEAR]
    y = y.loc[X.index]

    X_train, X_test  = X[~X['year'].isin(test_years)], X[X['year'].isin(test_years)]
    y_train, y_test  = y[X_train.index], y[X_test.index]
        
    return X_train, X_test, y_train, y_test, X_real, y_real

def get_mi_feature_importance(x, y,num_of_features):
    x=x.reset_index(drop=True)
    y=y.reset_index(drop=True)
    
    # Mutual Information 계산
    mi_scores = mutual_info_regression(x, y)

    # 중요 Feature만 선택
    feature_importances = pd.Series(mi_scores, index=x.columns)
    selected_features = feature_importances[feature_importances > feature_importances.mean()].index  # 임계값 설정
    
    # 모델별 중요도 시각화
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    for idx, importance in enumerate(feature_importances.items()):
        sorted_idx = np.argsort(importance)[::-1]  # 중요도 내림차순 정렬
        axes[idx].barh(x.columns[sorted_idx], importance[sorted_idx], color='skyblue')
        axes[idx].set_title(f'Mutual Information Feature Importance')
        axes[idx].set_xlabel('Importance')
        axes[idx].invert_yaxis()  # 중요도 높은 순서로 표시
    
    plt.tight_layout()
    plt.show()

    if num_of_features:
        selected_features = selected_features[:num_of_features]
        
    # pd.DataFrame(selected_features).to_csv("selected_features.csv",index=None,encoding='utf-8-sig')
    return selected_features

In [55]:
def my_data_load(num_of_features, test_years, group_idx):
    # 데이터 로딩
    bef = pd.read_csv("배추_transpose_2024.csv",encoding='utf-8-sig')
    aft = pd.read_csv("combined_weather_data(preprocessed_weather).csv",encoding='utf-8-sig')

    # drop 2025
    bef_from_2006 = bef[(bef['year']<2024)&(bef['year']>=2006)]
    aft_from_2006 = aft[(aft['year']<2024)&(aft['year']>=2006)]

    # data_origin.index = data_origin['Year']
    bef_real_live_data = bef_from_2006[bef_from_2006['year'] == LIVE_YEAR]
    aft_real_live_data = aft_from_2006[aft_from_2006['year'] == LIVE_YEAR]
    
    # data_origin = preprocess_data(data_origin) # 일부 열이름 변경 (통일)
       
    X_train, X_test, y_train, y_test, X_real, y_real = split_x_y(bef_from_2006, test_years)
    
    X_scaled_train, scaler = scale_data(X_train, is_fit=True)
    X_scaled_test, scaler = scale_data(X_test, is_fit=False, scaler = scaler)
    X_scaled_live, scaler = scale_data(X_real, is_fit=False, scaler = scaler)
    
    # Feature Importance 함수 호출 (2024는 y값이 없어서 제외)
    final_features = get_feature_importance(X_scaled_train, y_train, num_of_features) 
    print("\nFinal Selected Features1:", final_features)
    
    # selected_features_2 = get_mi_feature_importance(X_scaled_train, y_train, num_of_features)
    # print("\nFinal Selected Features2:", selected_features_2)
    
    #final_features = list(set(selected_features_1)|set(selected_features_2))
    #print("common final feats : ", final_features)
    
    pd.DataFrame(final_features).to_csv(f"selected_features{MODEL_NUM}({group_idx}).csv",index=None,encoding='utf-8-sig')
    
    # final_features = ['11월_김치수입','해당년도 11월 배추 경락가','해당년도 12월 배추 경락가','diff_해당년도 5월 배추 경락가_하순-해당년도 5월 배추 경락가_상순','diff_해당년도 7월 배추 경락가_하순-해당년도 7월 배추 경락가_중순','diff_해당년도 8월 배추 경락가_중순-해당년도 8월 배추 경락가_상순','diff_해당년도 9월 배추 경락가_상순-해당년도 8월 배추 경락가_상순','diff_해당년도 9월 배추 경락가_하순-해당년도 7월 배추 경락가_하순','diff_해당년도 10월 배추 경락가_하순-해당년도 10월 배추 경락가_상순','diff_해당년도 10월 배추 경락가_상순-해당년도 6월 배추 경락가_상순']
    
    X_scaled_train.index, X_scaled_test.index = X_train.index , X_test.index
    X_train, X_test, X_scaled_live = X_scaled_train[final_features], X_scaled_test[final_features], X_scaled_live[final_features]
    
    return X_train, X_test, y_train, y_test, X_scaled_live, y_real

import joblib 

def main(test_years, group_idx):
    num_of_features = 10
    X_train, X_test, y_train, y_test, X_live, y_fakelive = my_data_load(num_of_features, test_years, group_idx)
    
    train_data = pd.concat([X_train,y_train],axis=1)
    test_data = pd.concat([X_test,y_test],axis=1)
    live_data = pd.concat([X_live, y_fakelive],axis=1)
    
    live_data = live_data.dropna(subset=['year']) # 2025등 널값제거
    train_for_live = pd.concat([train_data,test_data],axis=0,ignore_index=True)
    
    print(train_data.columns)
    
    MODEL_PATH = 'C:/Users/황회선/Desktop/2025/데이터(기상)/asos_weather_province/result'
    
    # # 모델 학습 및 최적화
    model_name = MODEL_PATH + f"_{group_idx}"
    base_model = train_and_optimize_models(train_data, 15, 5, model_name) # 데이터, 만들어낼모델수(15), 앙상블할 top-k모델수(5), 모델이름 
        
    # # 모델 로딩 및 기본 설정
    base_model = joblib.load(model_name + ".pkl")
    
    # 예측 및 결과
    predictions_train, predictions_test, train_avg_error_rate, test_avg_error_rate = predict_with_models(base_model, train_data, test_data)

    print(f"TEST YEARS: {test_years}")
    print(f"Train Average Error Rate: {train_avg_error_rate:.2f}%")
    print(f"Test({test_years}) Average Error Rate: {test_avg_error_rate:.2f}%")

    predictions_live_train, predictions_live, train_for_live_avg_error_rate, live_avg_error_rate = predict_with_models(base_model, train_for_live, live_data)
    print(f"LIVE YEAR: {LIVE_YEAR}")
    print(f"Train for live Average Error Rate: {train_for_live_avg_error_rate:.2f}%")
    print(f"Live({test_years}) Average Error Rate: {live_avg_error_rate:.2f}%")
    print(f"predicted Live({test_years}) : {predictions_live}")

import time 

test_years = [2022,2023]
start_time = time.time()
main(test_years, 1)
print(f"Execution Time: {time.time() - start_time:.2f} seconds")

c:\Users\황회선\.conda\envs\pypy\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4992.768993333913, tolerance: 1103.641363738879
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\황회선\.conda\envs\pypy\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17179.80475584464, tolerance: 1103.641363738879
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\황회선\.conda\envs\pypy\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26051.147220289335, tolerance: 1103.641363738879
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\황회선\.conda\envs\pypy\lib\site-packages\sklearn\linear_model\_coordina

Lasso Selected Features: ['year', 'price', 'dayNewSnow_50110', 'dayNewSnow_50130', 'daySnow_46820', 'maxTa_46820', 'minHum_46820', 'minHum_50130', 'prep_50110', 'sumSunhr_50110', 'wind_46820', 'wind_50110']
Ridge Selected Features: ['year', 'price', 'dayNewSnow_50110', 'daySnow_46820', 'maxTa_46820', 'minHum_46820', 'minHum_50130', 'prep_50110', 'sumSunhr_50110', 'wind_46820', 'wind_50110', 'wind_50130']
LARS Selected Features: ['year', 'month', 'price', 'dayNewSnow_50110', 'minHum_46820', 'sumSunhr_50110', 'wind_46820']

Final Selected Features1: ['year', 'price', 'dayNewSnow_50110', 'daySnow_46820', 'maxTa_46820', 'minHum_46820', 'minHum_50130', 'prep_50110', 'sumSunhr_50110', 'wind_46820']
Index(['year', 'price', 'dayNewSnow_50110', 'daySnow_46820', 'maxTa_46820',
       'minHum_46820', 'minHum_50130', 'prep_50110', 'sumSunhr_50110',
       'wind_46820', 'next_1_price'],
      dtype='object')
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configurat

,Description,Value
0,Session id,11
1,Target,next_1_price
2,Target type,Regression
3,Original data shape,"(192, 11)"
4,Transformed data shape,"(192, 11)"
5,Transformed train set shape,"(153, 11)"
6,Transformed test set shape,"(39, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3080 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no mo

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,131.8933,31275.9019,176.8022,0.5148,0.3228,0.2847,0.1400
lightgbm,Light Gradient Boosting Machine,135.5215,33138.8563,181.7024,0.4946,0.3430,0.2947,0.1700
et,Extra Trees Regressor,130.0878,33628.5497,183.2544,0.4831,0.3143,0.2725,0.1200
huber,Huber Regressor,140.1432,34911.7994,185.8289,0.4690,0.3535,0.3074,0.0067
br,Bayesian Ridge,142.4103,35276.4596,187.2115,0.4609,0.3528,0.3193,0.0067
ridge,Ridge Regression,142.1274,35371.9909,187.3495,0.4599,0.3609,0.3204,0.0067
lr,Linear Regression,142.1346,35469.1439,187.5974,0.4582,0.3648,0.3210,0.0100
llar,Lasso Least Angle Regression,142.3957,35495.1035,187.6480,0.4580,0.3589,0.3199,0.0067
lasso,Lasso Regression,142.3980,35496.5911,187.6515,0.4580,0.3589,0.3199,0.0033
par,Passive Aggressive Regressor,141.9845,36391.3235,189.4878,0.4450,0.3518,0.3061,0.0067


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,67.0880,8160.5793,90.3359,0.8816,0.1695,0.1438


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,160.5503,62094.0900,249.1869,0.5146,0.2539,0.1899


TEST YEARS: [2022, 2023]
Train Average Error Rate: 14.38%
Test([2022, 2023]) Average Error Rate: 18.99%


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,72.5857,11333.1388,106.4572,0.8529,0.1756,0.1465


ValueError: 12 missing values found in the target column: next_1_price. To proceed, remove the respective rows from the data. 

In [ ]:

    # # base_model이 VotingRegressor일 경우, 각 모델에 랜덤 시드 설정
    # if isinstance(base_model, VotingRegressor):
    #     random_seed = np.random.randint(0, 100)  # 랜덤 시드
    #     estimators = base_model.estimators_  # VotingRegressor의 모델들
        
    #     # 각 모델에 대해 시드 설정
    #     for name, model in estimators:
    #         if hasattr(model, 'random_state') and model.random_state is None:
    #             model.random_state = random_seed
    #             print(f"Random seed set for {name}")

    # # 기존 `base_models_params` 생성 부분 수정
    # base_models_params = []
    # seen_names = set()

    # for i, estimator in enumerate(base_model.estimators_):
    #     # 튜플 형태인지 확인
    #     if isinstance(estimator, tuple) and len(estimator) == 2:
    #         name, model = estimator
    #     else:
    #         name = type(estimator).__name__
    #         model = estimator

    #     # 고유 이름 생성
    #     original_name = name
    #     counter = 1
    #     while name in seen_names:
    #         name = f"{original_name}_{counter}"  # 중복 시 숫자 추가
    #         counter += 1
    #     seen_names.add(name)

    #     # 고유 이름과 모델 추가
    #     base_models_params.append((name, model))

    # print(f"Base models with unique names: {[name for name, _ in base_models_params]}")
    
    # results = []

    # # 교차 검증 및 예측 반복
    # train_data['Year'] = train_data.index
    # test_data['Year'] = test_data.index
    # for n in tqdm(range(1, 9)):
    #     random_seed = np.random.randint(0, 100)
    #     for val, train in split_data(train_data):
    #         model = create_voting_regressor(base_models_params, random_seed)
    #         predictions_train, predictions_val, predictions_test = train_and_predict(
    #             model, train, val, test_data, TARGET_COL
    #         )
    #         # 에러율 계산
    #         error_rate_train = calculate_error_rate(train[TARGET_COL], predictions_train)
    #         error_rate_val = calculate_error_rate(val[TARGET_COL], predictions_val)
    #         error_rate_test = calculate_error_rate(test_data[TARGET_COL], predictions_test)
            
    #         # live 환경을 가정하여(2024) 추가된 데이터(2021,2022,2023)로 모델 학습
    #         live_data = pd.concat([train,test_data]).sort_index(ascending=False)
    #         model.fit(live_data.drop([TARGET_COL]+['Year'],axis=1), live_data[TARGET_COL])
    #         real_live_preds = model.predict(X_live)[0]

    #         error_rate_fakelive = 100*(y_fakelive-real_live_preds)/y_fakelive
            
    #         # 결과 저장
    #         results.append({
    #             "nth_model": n,
    #             "train": error_rate_train,
    #             "val": error_rate_val,
    #             f"test_preds_{test_years[0]}": predictions_test[0],
    #             f"test_preds_{test_years[1]}": predictions_test[1],
    #             f'test_error({test_years})': error_rate_test,
    #             "val_years": list(val['Year'].values),
    #             f'live_preds_{LIVE_YEAR}': real_live_preds,
    #             f'live_err' : error_rate_fakelive,
    #         })
    
    # # 결과 저장 및 분석
    # result_df = pd.DataFrame(results)
    # save_result_data(result_df, RESULT_PATH + f"_({group_idx}).csv")

    # # 최종 분석
    # analyzed_results = analyze_results(result_df, test_years, top_k=3) # 년도조합별, 실험내 val이 낮은 상위 k개 모델 예측결과 앙상블
    # print("최종 분석 결과:", analyzed_results)
